In [14]:
#!/bin/env python3
import matplotlib.pyplot as plt
from getfeatures import features, getfeature
import tensorflow as tf
import xml.etree.ElementTree as ET
import numpy
import matplotlib.pyplot as plt
numpy.set_printoptions(precision=2,threshold=1000,suppress=True)

print('Reading corpus and finding features')
xmlcorpus = ET.parse('../Corpus/all.xml')
nodoc = 0

print('Création de la matrice numpy pour x et y')
docs = xmlcorpus.getroot().getchildren()
featurekeys = sorted(list(features.keys()))
x = numpy.zeros((len(docs), len(featurekeys)))
y = numpy.zeros((len(docs),1))

print('Insertion des données dans la matrice')
for i in range(len(docs)):
    for j in range(len(featurekeys)):
        doc = docs[i]
        featurename = featurekeys[j]
        x[i,j] = getfeature(doc, featurename)
        fake = 0
        if doc.get('class') == 'fake':
            fake = 1
        y[i] = fake
print(x.shape)
print(y.shape)

Reading corpus and finding features
Création de la matrice numpy pour x et y
Insertion des données dans la matrice
(130, 16)
(130, 1)


In [2]:
def add_layer(inputs,in_size,out_size,activation_function=None):
    #随机生成Weights和biases
    Weights = tf.Variable(tf.random_normal([in_size,out_size]))
    biases = tf.Variable(tf.zeros([1,out_size]) + 0.1) 
    Wx_plus_b = tf.matmul(inputs,Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

In [18]:
xs = tf.placeholder(tf.float32,[None,len(featurekeys)])
ys = tf.placeholder(tf.float32,[None,1])


(?, 1)


In [23]:
l1 = add_layer(xs,len(featurekeys),10,activation_function=tf.nn.relu)
prediction = add_layer(l1,10,1,activation_function=tf.nn.sigmoid)

loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction),
                    reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    for i in range(1000):
        sess.run(train_step,feed_dict={xs:x,ys:y})
        if i % 50 ==0:
            print(sess.run(loss,feed_dict={xs:x,ys:y}))
            #print(sess.run(prediction,feed_dict={xs:x,ys:y}))


0.2619664
0.22760768
0.2274586
0.22732896
0.2272156
0.22711599
0.22702809
0.22695017
0.22686128
0.22677952
0.22670116
0.22663453
0.2265775
0.22652851
0.22648609
0.22644931
0.22641711
0.22638892
0.22636427
0.22634253
